# Notebook for a space-time discretization

In this notebook we are treating following problem 
$$
u_t - \Delta u + \mathbf{w} \cdot \nabla u = f \quad \text{in } \Omega  = [0,1] \times [0,T].
$$
with the initial/boundary conditions
$$
u(x,0) = \sin(\pi x), \quad u(0,t) = u(1,t)=0.
$$
The weakformulation of the problem in the CG setting is

Find a function $u_0 \in H^1_0(\Omega)$ , such that  for all  $v \in H^1_0(\Omega) $:

$$
\int_{\Omega} (u_t)_0 v \,dx \,dt
+ \int_{\Omega} \nabla u_0 \cdot \nabla v \,dx \,dt
+ \int_{\Omega} (\mathbf{w} \cdot \nabla u_0) v \,dx \,dt
= \int_{\Omega} f v \,dx \,dt
- \int_{\Omega} u_D v \,dx \,dt.
$$

where we used the decomposition of $u = u_D + u_0$

For $w = 0$ (the heatequation), we have for the chosen initial/boundary conditions following analytical solution 

$u(x,t)= \sin(\pi x) e^{-t \pi^2}$

In [2]:
from ngsolve import *
from netgen.geom2d import SplineGeometry
from ngsolve.webgui import Draw
from netgen.occ import *

p = -1
T = 2 # length of the time interval
shape = Rectangle(1,T).Face()
shape.edges.Min(X).name="left"
shape.edges.Max(X).name="right"
shape.edges.Min(Y).name="bottom"
shape.edges.Max(Y).name="top"
mesh = Mesh(OCCGeometry(shape, dim=2).GenerateMesh(maxh=0.1))
Draw(mesh)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [3]:
deformation = GridFunction(H1(mesh,order=1,dim=mesh.dim))
deformation.Set(CF((x,y/2))-CF((x,y)))
Draw(deformation,mesh)
mesh.SetDeformation(deformation)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

In [4]:
Draw(y,mesh,deformation=deformation)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [5]:
shape = Rectangle(1,1).Face()
shape.edges.Min(X).name="left"
shape.edges.Max(X).name="right"
shape.edges.Min(Y).name="bottom"
shape.edges.Max(Y).name="top"
mesh = Mesh(OCCGeometry(shape, dim=2).GenerateMesh(maxh=0.1))
Draw(y,mesh)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [18]:
def abs(u):
        return IfPos(u[1],u,-u)
def maxy(mesh,deformation):
        max_y_difference = 0  # Variable zur Speicherung der größten y-Differenz
        # Iteriere über alle Dreiecke im Mesh
        for el in mesh.Elements():
                vertices = [tuple(deformation(mesh[vi].point[0], mesh[vi].point[1])) for vi in el.vertices]  # Eckpunkte des Dreiecks
                y_coords = [p[0] for p in vertices]  # Nur die y-Koordinaten extrahieren

                y_difference = max(y_coords) - min(y_coords)  # Höchster - niedrigster Wert
                max_y_difference = max(max_y_difference, y_difference)  # Maximale Differenz speichern

        print(f"Maximaler Unterschied der y-Koordinaten: {max_y_difference}")

meshes =[]
for p in range(-3,1):
        for i in range(-3,1):
                L = 2**i
                T = 2**(p) # length of the time interval
                shape = Rectangle(L,T).Face()
                shape.edges.Min(X).name="left"
                shape.edges.Max(X).name="right"
                shape.edges.Min(Y).name="bottom"
                shape.edges.Max(Y).name="top"
                mesh = Mesh(OCCGeometry(shape, dim=2).GenerateMesh(maxh=0.05))
                deformation = GridFunction(H1(mesh,order=1,dim=mesh.dim))
                deformation.Set(CF((2**-i*x,2**(-p)*y))-CF((x,y)))
                mesh.SetDeformation(deformation)
                #Draw(sin(y),mesh,deformation=deformation)
                print(f"p = {p}" + f" i = {i}")       
                maxy(mesh,deformation=deformation)
                meshes.append(mesh)





p = -3 i = -3
Maximaler Unterschied der y-Koordinaten: 0.29557642599764333
p = -3 i = -2
Maximaler Unterschied der y-Koordinaten: 0.15000000000000036
p = -3 i = -1
Maximaler Unterschied der y-Koordinaten: 0.050000000000000155
p = -3 i = 0
Maximaler Unterschied der y-Koordinaten: 0
p = -2 i = -3
Maximaler Unterschied der y-Koordinaten: 0.45190086176408134
p = -2 i = -2
Maximaler Unterschied der y-Koordinaten: 0.1867313663709549
p = -2 i = -1
Maximaler Unterschied der y-Koordinaten: 0.06264326090835642
p = -2 i = 0
Maximaler Unterschied der y-Koordinaten: 0
p = -1 i = -3
Maximaler Unterschied der y-Koordinaten: 0.45346163187688865
p = -1 i = -2
Maximaler Unterschied der y-Koordinaten: 0.18374581432183476
p = -1 i = -1
Maximaler Unterschied der y-Koordinaten: 0.06286802543865669
p = -1 i = 0
Maximaler Unterschied der y-Koordinaten: 0
p = 0 i = -3
Maximaler Unterschied der y-Koordinaten: 0.4534649403449945
p = 0 i = -2
Maximaler Unterschied der y-Koordinaten: 0.18632907704361487
p = 0 i = 

In [17]:
w = 0
for mesh in meshes:
        fes = H1(mesh, order=1, dirichlet="left|right")
        gfu = GridFunction(fes)
        gfu.Set(exp(w*x/2)*sin(pi*x),BND)  # initial condition
        Draw(gfu)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

...

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…